In [1]:
import pandas as pd

In [2]:
df  = pd.read_csv('http://bit.ly/kaggletrain')

In [3]:
df.shape

(891, 12)

In [10]:
df.columns
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df = df.loc[df.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]

In [7]:
df.shape

(889, 4)

In [8]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [9]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


## Step 3: Cross-validate a model with one feature

In [12]:
X= df.loc[:,['Pclass']]
y= df.Survived

In [13]:
X.shape

(889, 1)

In [14]:
y.shape

(889,)

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
logreg = LogisticRegression()

In [17]:
from sklearn.model_selection import cross_val_score

In [18]:
cross_val_score(logreg,X,y,cv=5,scoring='accuracy').mean()

0.6783406335301212

In [19]:
y.value_counts(normalize=True)

Survived
0    0.617548
1    0.382452
Name: proportion, dtype: float64

In [20]:
# Step4: Encode categorical features 

In [21]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [22]:
# dummy encoding of categorical features
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [23]:
ohe.fit_transform(df[['Sex']])

C:\Users\Aditya\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [24]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [25]:
ohe.fit_transform(df[['Embarked']])

C:\Users\Aditya\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [26]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

### Cross-validate a Pipeline with all features

In [28]:
X= df.drop('Survived',axis='columns')
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [29]:
from sklearn.compose import make_column_transformer

column_trans = make_column_transformer((OneHotEncoder(),['Sex','Embarked']),remainder='passthrough')

In [30]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [31]:
from sklearn.pipeline import make_pipeline

In [33]:
pipe = make_pipeline(column_trans,logreg)

In [34]:
# Cross validate the entire process
# thus, preprocessing occurs within each fold of cross-validation

cross_val_score(pipe,X,y,cv=5,scoring ='accuracy').mean()

0.7727924839713071

### Make predictions on "new " data

In [35]:
# added empty cell so that the cell numbering matches the video

In [36]:
X_new = X.sample(5,random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [37]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [38]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)

## Recap

In [39]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [41]:
df= pd.read_csv('http://bit.ly/kaggletrain')
df = df.loc[df.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]
X = df.drop('Survived',axis='columns')
y= df.Survived

In [42]:
column_trans = make_column_transformer(
(OneHotEncoder(),['Sex','Embarked']),remainder = 'passthrough')

logreg = LogisticRegression(solver='lbfgs')

In [43]:
pipe = make_pipeline(column_trans,logreg)

In [44]:
cross_val_score(pipe,X,y,cv=5,scoring='accuracy').mean()

0.7727924839713071

In [45]:
X_new = X.sample(5,random_state=99)

In [46]:
pipe.fit(X,y)
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)